In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import re
import requests
pd.options.display.max_rows = 5000 

In [2]:
pwd

'/Users/paxton615/Github_Personal/Notes/Notes/Financial_models/SI Stocks'

- Stock data: 

spaq : Spinaker Aquisitions

In [3]:
url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/hi/history/spaq.l/1d"

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com"
    }

spaq = requests.request("GET", url, headers=headers)

# print(response.text[:500])

- Stock data

pre : Pensana

In [4]:
url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/hi/history/pre.l/1d"

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com"
    }

pre = requests.request("GET", url, headers=headers)

In [5]:
spaq = spaq.text
pre = pre.text

In [6]:
def get_dates(string):
    date = re.compile('"date":((.*),)')
    date = date.findall(string)
    date = [i[0][2:12] for i in date]
    date = [datetime.strptime(i, "%d-%m-%Y").strftime("%Y-%m-%d") for i in date]
    return date

def get_close(string):
    close = re.compile('"close":((.*),)')
    close = close.findall(string)
    close = [float(i[1].strip()) for i in close]
    return close

In [7]:
spaq_date = get_dates(spaq)
spaq_close = get_close(spaq)

In [8]:
pre_date = get_dates(pre)
pre_close = get_close(pre)

In [9]:
print(spaq_date[-1])
print(spaq_close[-1])
print(pre_date[-1])
print(pre_close[-1])

2021-08-17
15.5
2021-08-18
83.8


In [10]:
fund = 10000
client = ['Ma', 'Ma']
stock = ['spaq', 'pre']
total_cap = [10000, 10000]
invest_in = [5000, 5000]
ipo_price = [0.1, 1.34]
w_pirce = [0.2, 0]
s_position = pd.Series(invest_in)/pd.Series(ipo_price)
p_vs_w = [0.5, 0]
w_position = pd.Series(s_position)*pd.Series(p_vs_w)
s_date = [spaq_date[-1], pre_date[-1]]
s_close = [spaq_close[-1]/100, pre_close[-1]/100]

In [11]:
mr_ma = pd.DataFrame({'client':client, 'stock':stock, 'total_cap':total_cap, 'invest_in':invest_in, \
             'ipo_price':ipo_price, 'w_pirce':w_pirce ,'p_vs_w':p_vs_w,'s_position':s_position, \
             'w_position':w_position, 's_date':s_date, 's_close':s_close,\
             })

In [12]:
mr_ma['s_pft'] = (mr_ma['s_close'] - mr_ma['ipo_price'])* mr_ma['s_position']

mr_ma['w_pft'] = [(i-j)*k if i>j else 0 for i,j,k in zip(mr_ma.s_close, mr_ma.w_pirce, mr_ma.w_position)]

mr_ma['charge_plcing'] = 0.03 * mr_ma['invest_in'] + 10

In [13]:
mr_ma

,client,stock,total_cap,invest_in,ipo_price,w_pirce,p_vs_w,s_position,w_position,s_date,s_close,s_pft,w_pft,charge_plcing
0,Ma,spaq,10000,5000,0.10,0.2,0.5,50000.000000,25000.0,2021-08-17,0.155,2750.000000,0.0,160.0
1,Ma,pre,10000,5000,1.34,0.0,0.0,3731.343284,0.0,2021-08-18,0.838,-1873.134328,0.0,160.0


In [15]:
account_balance = fund + sum(mr_ma['s_pft'])+sum(mr_ma['w_pft'])

total_charges = sum( mr_ma['charge_plcing'])

total_pft = account_balance - total_charges

In [16]:
print('Account_balance:', account_balance)
print('total_charges:', total_charges)
print('total_profit:', total_pft)

Account_balance: 10876.86567164179
total_charges: 320.0
total_profit: 10556.86567164179


In [25]:
import requests

url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {"interval":"60min","function":"TIME_SERIES_INTRADAY","symbol":"MSFT","datatype":"json","output_size":"compact"}

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "alpha-vantage.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{
    "Meta Data": {
        "1. Information": "Intraday (60min) open, high, low, close prices and volume",
        "2. Symbol": "MSFT",
        "3. Last Refreshed": "2021-08-18 20:00:00",
        "4. Interval": "60min",
        "5. Output Size": "Compact",
        "6. Time Zone": "US/Eastern"
    },
    "Time Series (60min)": {
        "2021-08-18 20:00:00": {
            "1. open": "290.6000",
            "2. high": "290.7900",
            "3. low": "290.3600",
            "4. close": "290.7500",
            "5. volume": "13117"
        },
        "2021-08-18 19:00:00": {
            "1. open": "290.3500",
            "2. high": "290.7000",
            "3. low": "290.3500",
            "4. close": "290.5900",
            "5. volume": "8762"
        },
        "2021-08-18 18:00:00": {
            "1. open": "290.1701",
            "2. high": "290.7300",
            "3. low": "290.0200",
            "4. close": "290.2500",
            "5. volume": "26231"
        },
        "2021-08-18

In [26]:
import requests

url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {"function":"TIME_SERIES_DAILY","symbol":"MSFT","outputsize":"compact","datatype":"json"}

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "alpha-vantage.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{
    "Meta Data": {
        "1. Information": "Daily Prices (open, high, low, close) and Volumes",
        "2. Symbol": "MSFT",
        "3. Last Refreshed": "2021-08-18",
        "4. Output Size": "Compact",
        "5. Time Zone": "US/Eastern"
    },
    "Time Series (Daily)": {
        "2021-08-18": {
            "1. open": "292.0400",
            "2. high": "294.8200",
            "3. low": "290.2700",
            "4. close": "290.7300",
            "5. volume": "21812985"
        },
        "2021-08-17": {
            "1. open": "292.3900",
            "2. high": "293.4300",
            "3. low": "291.0800",
            "4. close": "293.0800",
            "5. volume": "20075250"
        },
        "2021-08-16": {
            "1. open": "293.1900",
            "2. high": "294.8200",
            "3. low": "290.0200",
            "4. close": "294.6000",
            "5. volume": "22516027"
        },
        "2021-08-13": {
            "1. open": "289.4800",
            "2. high": "29

In [36]:
import requests

url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-historical-data"

querystring = {"symbol":"avo.l","region":"uk"}

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"prices":[{"date":1629270000,"open":40,"high":41,"low":38.29999923706055,"close":39,"volume":98562,"adjclose":39},{"date":1629183600,"open":40,"high":41,"low":39.266998291015625,"close":40,"volume":177542,"adjclose":40},{"date":1629097200,"open":39.5,"high":40.474998474121094,"low":38,"close":40,"volume":289653,"adjclose":40},{"date":1628838000,"open":37.5,"high":40.7400016784668,"low":37.79999923706055,"close":39,"volume":450079,"adjclose":39},{"date":1628751600,"open":38,"high":39.79999923706055,"low":37.150001525878906,"close":37.5,"volume":301532,"adjclose":37.5},{"date":1628665200,"open":37.5,"high":40,"low":36.599998474121094,"close":38.5,"volume":647949,"adjclose":38.5},{"date":1628578800,"open":35.5,"high":36.45000076293945,"low":34,"close":35.5,"volume":66885,"adjclose":35.5},{"date":1628492400,"open":35.5,"high":37.45800018310547,"low":34.33300018310547,"close":35.5,"volume":111417,"adjclose":35.5},{"date":1628233200,"open":35.5,"high":36.650001525878906,"low":34.33300018310

In [28]:
from datetime import datetime

In [40]:
datetime.fromtimestamp(1629270000).isoformat()

'2021-08-18T08:00:00'